# py0xcluster

This is for now an educational project for myself with the aim of performing fun data-science projects around blockchain data gathered through the Graph Network (https://thegraph.com/)

## Target objectives

- Establish meaningful grouping of address by clustering DEX traders and LP
    - Feature Extraction:
        - TBD but based on mint/swap/burn data from messari subgraphs entities
        - with or without balances at swap time (web3py fetch balance at block)
        - EOA vs Contracts
    - Dimensionality reduction:
        - UMAP / tSNE or PCA / ICA
    - Clustering:
        - DBSCAN
        - silhouette evaluation
    - Visualization:
        - scatter plot with color-coded returns? (TBD)

**Secondary objective: identify which group has the most profitable activity**

- Triggered Average of price by swap in/out by group of addresses

- Predict future returns based on the activity of previously clustered groups of addresses

## Random list of potential features:

### Addresses
- z-scored (clarify how) difference of price 24h? after swap -> could be target independant variable
- % of Limit order on uni-v3 (one deposit amout = 0)
- is contract
- nb of events (z-scored to other addresses on same pool)
- average swap size (z-scored/pool)
- average deposit size (z-scored/pool)
- average withdraw size (z-score/pool)
- ratio? of nb of: swaps / (deposits + withdraws)
- nb of liquid pools interacted with
- % of events (likely swaps) happening in the same block (possibly identical to MEV bots?)




### Imports

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from py0xcluster.utils.query_utils import *
from py0xcluster.main_classes.pools import *
from py0xcluster.main_classes.pool_events import *

### Gathering data about most-active pools
    - need to adapt to take into account refactoring of queries-related methods for multiple entities at once

In [4]:
uni3pools_selector = PoolSelector(
    subgraph_url = 'https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum',
    min_daily_volume_USD = 100000,
    min_TVL = 100000, # Not implemented. consider removing
    start_date = (2022,12,21), 
    end_date = (2023,1,10),
    days_batch_size = 20)

uni3_pools = uni3pools_selector.create_pool_selection(verbose=True)
uni3_pools.pools_df

Queriying from 2022-12-21 00:00:00 to 2023-01-10 00:00:00
2650 lquidity pools snapshots retrieved
357 stable pools snapshots (over 2650) have been removed
274 illiquid pools snapshots (over 2650) have been removed 
221 pools were selected


,pool.name,dailyVolumeUSD,pool.totalValueLockedUSD,token0.lastPriceUSD,token1.lastPriceUSD,pool.protocol.name,pool.protocol.network,pool.id,token0.symbol,token1.symbol
0,Uniswap V3 USD Coin/Wrapped Ether 0.05%,1.407573e+08,1.844735e+08,1.000000,1419.860391,Uniswap V3,MAINNET,0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640,USDC,WETH
1,Uniswap V3 USD Coin/Tether USD 0.01%,7.632963e+07,1.199310e+08,1.000000,1.000000,Uniswap V3,MAINNET,0x3416cf6c708da44db2624d63ea0aaef7113527c6,USDC,USDT
2,Uniswap V3 Wrapped Ether/Tether USD 0.05%,2.748191e+07,1.812181e+07,1419.860391,1.000000,Uniswap V3,MAINNET,0x11b815efb8f581194ae79006d24e0d814b7697f6,WETH,USDT
3,Uniswap V3 USD Coin/Wrapped Ether 0.01%,1.951618e+07,2.979251e+06,1.000000,1419.860391,Uniswap V3,MAINNET,0xe0554a476a092703abdb3ef35c80e0d76d32939f,USDC,WETH
4,Uniswap V3 Wrapped BTC/Wrapped Ether 0.05%,1.490067e+07,1.196053e+08,18804.772271,1419.860391,Uniswap V3,MAINNET,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,WBTC,WETH
...,...,...,...,...,...,...,...,...,...,...
216,Uniswap V3 Green/Wrapped Ether 0.05%,1.030147e+05,6.334998e+05,0.002009,1419.860391,Uniswap V3,MAINNET,0x4efc9e2e3e77732ce2f9612b8f050082c01688bd,GREEN,WETH
217,Uniswap V3 HuntToken/Tether USD 0.3%,1.028683e+05,1.019830e+06,0.252896,1.000000,Uniswap V3,MAINNET,0x54578b6f942aeb23b67a8cef24220651306b8e26,HUNT,USDT
218,Uniswap V3 Index/Wrapped Ether 1%,1.026767e+05,2.350162e+04,0.000000,1419.860391,Uniswap V3,MAINNET,0x8c13148228765ba9e84eaf940b0416a5e349a5e7,INDEX,WETH
219,Uniswap V3 unification.com/xfund/Wrapped Ether 1%,1.023294e+05,1.365642e+04,0.000000,1419.860391,Uniswap V3,MAINNET,0xb1223da8a5929bcfa9d26f0c6da8f0a29c3925ff,xFUND,WETH


## Extracting all events from these pools

### Steps

- Data query:
    - Perform query for each pool, batch by days to accomodate response limit
    - Alternatively the query can be done on multiple pools. Ideally, the size of the batch should be proportional to the volume, but quite arbitrary to implement.
    - Consider whether do swaps / mints / burns separately or jointly
    - Loop and aggregate over days / pools (or batch of pools)


In [6]:
uni3_events_getter = PoolEventGetter(
    subgraph_url = 'https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum',
    pool_ids = uni3_pools.pools_df['pool.id'][1:5],
    start_date = (2023,1,1), 
    end_date = (2023,1,11),
    days_batch_size = 5
    )

# Get swaps, deposits, and withdraw from a pool.
uni3_events = uni3_events_getter.get_events(verbose=True)


pool: 0x3416cf6c708da44db2624d63ea0aaef7113527c6
Queriying from 2023-01-01 00:00:00 to 2023-01-06 00:00:00
Queriying from 2023-01-06 00:00:00 to 2023-01-11 00:00:00
pool: 0x11b815efb8f581194ae79006d24e0d814b7697f6
Queriying from 2023-01-01 00:00:00 to 2023-01-06 00:00:00
variables:
{'pool_id': '0x11b815efb8f581194ae79006d24e0d814b7697f6', 'start_date': 1672527600, 'end_date': 1672959600, 'first': 1000, 'skip': 6000}
Queriying from 2023-01-06 00:00:00 to 2023-01-11 00:00:00
variables:
{'pool_id': '0x11b815efb8f581194ae79006d24e0d814b7697f6', 'start_date': 1672959600, 'end_date': 1673391600, 'first': 1000, 'skip': 6000}
pool: 0xe0554a476a092703abdb3ef35c80e0d76d32939f
Queriying from 2023-01-01 00:00:00 to 2023-01-06 00:00:00
variables:
{'pool_id': '0xe0554a476a092703abdb3ef35c80e0d76d32939f', 'start_date': 1672527600, 'end_date': 1672959600, 'first': 1000, 'skip': 6000}
Queriying from 2023-01-06 00:00:00 to 2023-01-11 00:00:00
variables:
{'pool_id': '0xe0554a476a092703abdb3ef35c80e0d76d3

In [7]:
uni3_events['swaps']

,amountInUSD,amountOutUSD,amountIn,amountOut,timestamp,blockNumber
0,30.3001949429576779806272128378236,30.29800228304182014549789542381334,181536,24923144600597858,1672749587,16326315
1,8477.653701441760237818797661916639,8463.156329137829703090463911929758,7069381758526999800,51124323,1672638863,16317129
2,49.86275997430547879583375198512306,49.93802135465865707620807048082406,297413,39940122981501086,1672908683,16339513
3,3897.178510976839710745838444871377,3894.97703458905455672666946873099,3207124253084637119,23345151,1672701071,16322288
4,10255.96483594409789826901975763386,10266.17361028072757014051390016594,8084410571425783085,60344138,1672858415,16335342
...,...,...,...,...,...,...
9170,58.65948379663715203125332725234445,58.54522260272434570021549319752193,44081344541618760,338527,1673374739,16378154
9171,13.17306596745485508320704856426179,13.20300889663099466126193955083321,10000000000000000,76308,1673257463,16368426
9172,26.38692326464516295088497560549854,26.38719754431123016886651141421227,20000000000000000,153539,1673308967,16372692
9173,20290.83518789971343171646989465041,20296.43033644063872389173546852674,15987142830843751677,119600363,1673195375,16363268


In [8]:
uni3_events['deposits']

,amountUSD,from,to,inputTokenAmounts,timestamp,blockNumber
0,7327.485058766932061996012164391359,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,"[24159789, 2772549080237033650]",1672614299,16315087
1,9976.129386023189766835039757821674,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,"[31686341, 3699999998760819316]",1672822835,16332391
2,78647.62345792383552180827306636028,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,"[253118754, 30828623009319230697]",1672583831,16312560
3,144671.1029878807976746179357940413,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,"[579217804, 37726697025029579766]",1672803899,16330818
4,19974.3380261167445647869205285875,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,"[63049944, 7470124867161922515]",1672841027,16333900
...,...,...,...,...,...,...
247,186653.5096420824461123502435830993,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,"[150000000, 127642869547079167294]",1673025587,16349190
248,190390.7308455182047492074877053182,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,"[353461171, 96499999794997528174]",1673279819,16370282
249,6708.237397128633238281498484562213,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,"[19636505, 2499999945377070893]",1673263679,16368944
250,208267.6393525901136017608382232342,0xc36442b4a4522e871399cd717abdd847ab11fe88,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,"[582655719, 86986728635045580535]",1673153003,16359757


In [9]:
uni3_events['withdraws']

,amountUSD,from,to,inputTokenAmounts,timestamp,blockNumber
0,301.370906442093612580722573371647,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,0xc36442b4a4522e871399cd717abdd847ab11fe88,"[912938, 125170845300822884]",1672603391,16314182
1,1577064.008593653693445942496704291,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,0x56178a0d5f301baf6cf3e1cd53d9863437345bf9,"[3960158115, 751302679453054534569]",1672661279,16318988
2,130671.3884008317810810720637892081,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,0xc36442b4a4522e871399cd717abdd847ab11fe88,"[774150306, 90442585645143620]",1672833863,16333306
3,311.881984684025699408723199254181,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,0xb154af72c7f5760a92c46193aeb24ff8ddd64488,"[959922, 129001564732433855]",1672560443,16310619
4,1408034.502012494965827572410003714,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,0x56178a0d5f301baf6cf3e1cd53d9863437345bf9,"[5929493727, 355619893556157836626]",1672637843,16317044
...,...,...,...,...,...,...
307,199082.2338071125398624565635946906,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,0xc36442b4a4522e871399cd717abdd847ab11fe88,"[1148868595, 0]",1673279435,16370250
308,1534.891294226053850390792366865138,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,0xc36442b4a4522e871399cd717abdd847ab11fe88,"[4532485, 627713112039173213]",1673006411,16347603
309,10051173.719832926378804996241099,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,0x56178a0d5f301baf6cf3e1cd53d9863437345bf9,"[761130873, 7680644060829841202529]",1673227091,16365902
310,361056.545143419290338507992024772,0x4585fe77225b41b697c938b018e2ac67ac5a20c0,0xc36442b4a4522e871399cd717abdd847ab11fe88,"[2080832302, 1516114892846638132]",1673253143,16368068


In [ ]:
data_lengths = [0, 0, 0]
empty_data = [data_length == 0 for data_length in data_lengths]
all(empty_data)

True

In [ ]:
dico = {'ac': 0 , 'asfd': 2}
len(dico)

2